# Developer Notes
This notebook provides additional more detailed documentation and references to help a developer 
explore optional generalizations and extensions of the Streaming Analytics for Life Event prediction project. 

Areas of extension for this project include : 
* Change the logic/rules which select which new scores are considered significant
* Connect to other event sources , and data stores
* Modify the notebooks to perform streaming analytics with other artificial intelligence / machine learning scoring models
   * A special case of this occurs when the model being used is retrained and the new model has to be copied over for use in streaming analytics 

## Changing the Significant Event logic
In 'Streaming Analytics for Life Event Prediction', we pick out *significant score change events* and publish these to an IBM Event Streams topic where they could be used to trigger automated  near real-time proactive responses. The significant event selection logic is defined in the streaming-analytics-lfe-pipeline notebook in: Class SignificantEvents.

The current implementation of that class uses a high threshold (default value=0.5) and a low threshold (default value=0.4).
A new score is considered significant if: 
*  this is the first score for this customer above the high threshold since one or more earlier scores below the low threshhold  
*  or, this is the first score for this customer below the low threshold since one or more earlier scores above the high threshold for this customer. 
These rules for significant new scores are intended to capture when a customer is detected to be in a new state (where some novel policy action may be appropriate) and by requiring a threshold to be crossed, avoid noise and too much automated activity if a customer's scores are bouncing around a single threshold value. 

This significant score selection logic was picked to illustrate continuous monitoring of scores as they occur.  Any actual streaming analytics application will need its own specific significant score selection rules meeting the business needs of its context. The interested user should consider modifying the python functions in Class Significant Events and resubmitting the streaming_analytics_lfe_pipeline notebook to start a new streaming analytics simulation with modified rules for significant events.  


## Streaming Analytics with live event input - alternate score repositories 
An ideal demonstration of the value of streaming analytics might show continuous scoring using a well defined machine learning model with known business value on a live stream of arriving events.  For the Life Event Prediction model we have used, there is no public live stream reporting continuous interactions of customers with a financial services firm as they occur.  Therefore, in The Streaming Analytics for LFE project, we have used a simulated "live" stream generated from some historical data.  The sample input files are included in the source datasets provided with the project. 

An important extension step for a user of this accelerator would be adapt the input to receive an appropriate stream of live events. 
A high level view of the pattern to accomplish this is provided in the Publish significant events cell of the streaming-analytics-lfe-pipeline.  In the function createPublishSignificantEventsTopology() 
*  credentials to access an IBM Event Streams topic are retrieved
*  a publish command is issued to push an event to the topic

To work with a live input stream form a IBM Event Streams / Kafka one would use a similar credentials scheme and a subscribe command to the relevant input event topic rather than a publish. 

Output scores form the Streaming Analytics Are written to a relational database. Steps to configure an alternate database repository for scores and to connect to it are described in: https://github.com/IBM-ICP4D/IndustryAccelerators-RemoteData

## Streaming Analytics with retrained or other ML models 

###  Streaming with retrained ML models requires copying and moving  
This "Streaming Analytics for Life Event Prediction" accelerator uses the scoring functions of two machine learning(ML) models
from the Life Event Prediction Industry accelerator delivered with IBM Cloud Pak for Data version 2.1.0.0. The Life Event 
Prediction accelerator explains how to:
*   train the two ML models (Home Purchase and Relocation)  using a sample labelled dataset of customer interactions 
*   score specific customer interaction histories with the two models 

This accelerator shows how to deploy the resulting models to generate Home purchase and relocation probability scores
for a sample stream of continuously arriving customer interaction reports *as they occur* - using the streaming analytics capability of 
IBM Cloud Pak for Data.  

A feature of machine learning models is that their accuracy can be improved by periodically retraining and rebuilding them
with larger, more accurate, more recent or more reliable sets of labelled training data. 

The cells below describe the steps in moving an updated machine learning model created using the methods of the Life Event Prediction
 Accelerator into this Streaming Analytics For LFE accelerator project.  After the model is copied over, the updated model can be 
 deployed in streaming analytics mode by executing the streaming-analytics-lfe-pipeline notebook of this accelerator. That will start
 a new streaming analytics simulation using the updated model.  

We describe the model steps in the context of moving an updated LFE model and deploying it in streaming analytics model. 
This approach is general and can in principle be used as a pattern to deploy other Spark Python ML models in streaming analytics mode
using the Steam processing function of IBM Cloud Pak for Data.  



###  Concepts - background for scoring function update 
#### Runtime components used for scoring.
Analytics projects in IBM Cloud Pak for Data have a models subdirectory where all files needed to score new inputs are saved. 
When the Streaming Analytics for LFE project is created from a .tar file or git repository, the models subdirectory contains
initial trained versions of:
* LFE_HOME_PURCHASE_Model 
* LFE_RELOCATION_Model. 
These are the two models used in the Streaming Analytics for LFE accelerator. 

Updating the streaming analytics to use new models will require copying over updated model directories from where they have 
been created/retrained in a Life Event Prediction analytics project into this subdirectory of the Streaming Analytics for LFE project.  

#### Pickling the model
The Streams application that uses the model to score customer histories is composed in the 
streaming-analytics-lfe-pipeline Jupyter notebook in the function createScoreEventsTopology.

This topology object and all of its supporting components are bundled up and sent to a build server
for compilation as a Streams Job. This occurs in the line: 
sr4 = executor.submit(submitToStreams,createScoreEventsTopology()). 

This statement submits the scoring job to the streams instance. The submitted job will process
arriving customer interaction histories and generate using the model versions included in it.

In the process of composing the application the model is loaded and subsequently pickled. 
The model is unpickling on the Streams instance. Pickling/unpickling is a binary
serialization/deserialization process that dependent on the Python version.

#### Model support file.
The models are loaded and pickled by the notebook. Each model has
a metadata.json file embedded within it that is accessed during
the scoring process. During the steams scoring job composition, this file is moved
to file location where it is readable by the application. 


###  Copying / moving versions of the Life Event Prediction model
After models in the IndustryAccelerators-LifeEventPrediction project have been retrained and updated, they need to be copied over to the 'IndustryAccelerations-StreamingAnalytics' project for use in a streaming analytics deployment.

If the source and destination projects are installed on the same Cloud Pak for Data instance, all versions of the model
are to be copied and both source and destination projects are configured to use the same version of Scikit Learn, this can 
be done with a directory copy operation. 

1. Launch a Terminal command window in the 'IndustryAccelerators-StreamingAnalytics' project
2. For safety, if a model is being replaced, delete the current model folder in the project. 
      * For example, deleting LFE_HOME_PURCHASE_Model with: rm -r models/LFE_HOME_PURCHASE_Model
3.  Copy the new model folder from the  source Life Event Accelerator.
      * cp -r ../IndustryAccelerator-LifeEventPrediction/models/LFE_HOME_PURCHASE_Model models/LFE_HOME_PURCHASE_Model

There is a metadata.json file in the model directory which is aware of different numbered versions of the model.
Copying over a complete model directory will not be a problem. 

If the source and target projects ( Life Event Prediction and Streaming Analytics) are on different Cloud Pak for Data
instance an export step followed by an import step can be used to copy models over. 

When you have moved a new model into the IndustryAccelerations-StreamingAnalytics project. 
Executing the streaming_analytics_lfe_pipeline notebook will rebuild and submit
the Streams application with the updated models.

###  Reusing the Streaming Analytics pattern with ML models other than Life Event Prediction 

In extending the Streaming Analytics accelerator to use other ML models trained in a separate project, the following issues must be addressed. 

*  Since the data preparation step in ML model training is model specific, you will need to copy the prep python script from the training project into the streaming analytics deployment project. 
*  Any objects created during the training step in the source model development will also need to be copied into the streaming analytics deployment project. 
   * For example, the training_user_inputs.joblib and any json files need to be copied into the datasets folder of the streaming analytics project.
*  The scoring pipeline in the streaming analytics project needs to be updated to call the new prep script and class with new inputs specified.

There may also be code in the scoring pipeline scripts that is unique to the ML model being deployed. This would need to be reconciled regardless of whether training and deployment are done in separate projects or a single Cloud Pak for Data project.

## References
This section provides additional links to background IBM Streams and IBM Event Streams documentation. 

IBM Event Streams links :

    https://cloud.ibm.com/docs/services/EventStreams?topic=eventstreams-kafka_using
    https://github.com/ibm-messaging/event-streams-samples
    https://github.com/ibm-messaging/event-streams-samples/blob/master/kafka-python-console-sample/docs/Local.md
    https://docs.confluent.io/current/clients/confluent-kafka-python/
    Python toolkit: https://pypi.org/project/streamsx.eventstreams/
    IBM Event Streams (docs) - https://cloud.ibm.com/docs/services/EventStreams?topic=eventstreams-getting_started
    https://github.com/confluentinc/confluent-kafka-python
    https://docs.confluent.io/current/clients/confluent-kafka-python/

IBM Streams links

    https://streamsxtopology.readthedocs.io/en/latest/
    https://pypi.org/search/?q=streamsx



**This project contains Sample Materials, provided under license.  
Licensed Materials - Property of IBM.  
© Copyright IBM Corp. 2019. All Rights Reserved.  
US Government Users Restricted Rights - Use, duplication or disclosure restricted by GSA ADP Schedule Contract with IBM Corp.**
